In [31]:
import pandas as pd
data = pd.read_csv("../data/automobile_data.csv")
backup_data = pd.read_csv("../data/automobile_data.csv")

Check for null values in our data

In [32]:
print(data.isnull().sum())

Title                1
Price                1
Manufacturer         1
Category             1
Fuel Type            1
Fiscal Horsepower    1
Transmission         1
Mileage              1
Year                 1
Insertion Date       1
dtype: int64


Because we found that only one row is null in our initial dataset we can just drop it

In [33]:
data = data.dropna()

Double check for null values again

In [34]:
print(data.isnull().sum())

Title                0
Price                0
Manufacturer         0
Category             0
Fuel Type            0
Fiscal Horsepower    0
Transmission         0
Mileage              0
Year                 0
Insertion Date       0
dtype: int64


Replace white spaces between words with "_" in columns names for consistency

In [35]:
data.columns = data.columns.str.replace(" ", "_").str.lower()

Ensuring uniformity for categorical columns like fuel_type, transmission ... by checking unique vqlues and stardizing them

In [36]:
print(data["fuel_type"].unique())

['Essence' 'Diesel' 'Hybride essence' 'Hybride rechargeable essence'
 'Hybride rechargeable diesel' 'Hybride diesel' 'Electrique']


In [37]:
data["fuel_type"] = data["fuel_type"].replace({
    "Essence": "Petrol",
    "Hybride essence": "Hybrid Petrol",
    "Diesel": "Diesel",
    "Hybride rechargeable essence": "Hybrid Petrol",
    "Hybride rechargeable diesel": "Hybrid Diesel",
    "Hybride diesel": "Hybrid Diesel",
    "Electrique": "Electric"
})

In [38]:
print(data["transmission"].unique())

['Automatique' 'Manuelle']


In [39]:
data["transmission"] = data["transmission"].replace({
    "Automatique": "Automatic",
    "Manuelle": "Manual"
})

In [40]:
print(data["fuel_type"].unique())
print(data["transmission"].unique())

['Petrol' 'Diesel' 'Hybrid Petrol' 'Hybrid Diesel' 'Electric']
['Automatic' 'Manual']


Next we want to check for rows with unrealistic mileage (0 km) because we are working with only used cars, so the 0 km cars are not new that's why we will replace the 0 values with the average milage for used cars according to our dataset

In [41]:
print(data[data["mileage"] == 0])

                                          title     price manufacturer  \
6            Mercedes-Benz GLC Coupé 200 hybrid  450000.0     Mercedes   
64        Mercedes-Benz GLC Coupé GLC 200 COUPE  440000.0     Mercedes   
91   Land Rover Range Rover Evoque R dynamic SE  358000.0   Land Rover   
97     Land Rover Range Rover Sport HSE DYNAMIC  748000.0   Land Rover   
757                             Cupra Formentor  190000.0        Cupra   
909                 Mercedes-Benz GLE Coupé AMG  590000.0     Mercedes   

    category      fuel_type  fiscal_horsepower transmission  mileage     year  \
6    SUV/4x4  Hybrid Petrol               11.0    Automatic      0.0   1.2024   
64   SUV/4x4  Hybrid Petrol               11.0    Automatic      0.0   9.2023   
91   SUV/4x4         Diesel                9.0    Automatic      0.0   2.2024   
97   SUV/4x4  Hybrid Petrol               28.0    Automatic      0.0   2.2024   
757  SUV/4x4         Petrol               11.0    Automatic      0.0  11.202

In [42]:
median_mileage = data.loc[data["mileage"] > 0, "mileage"].median()
data["mileage"] = data["mileage"].replace(0, median_mileage)

In [43]:
print(data[data["mileage"] == 0]) 

Empty DataFrame
Columns: [title, price, manufacturer, category, fuel_type, fiscal_horsepower, transmission, mileage, year, insertion_date]
Index: []


Next, we are dropping duplicates from our data

In [44]:
data = data.drop_duplicates()

In [45]:
print(f"Number of rows after dropping duplicates: {len(data)}")

Number of rows after dropping duplicates: 1578


So we are left with 1578 after dropping duplicates, which is a good starting point to build upon. Let's preview our dataset after these changes for a final check before moving forward to the next steps

In [46]:
print(data.isnull().sum())

print(data.head())

title                0
price                0
manufacturer         0
category             0
fuel_type            0
fiscal_horsepower    0
transmission         0
mileage              0
year                 0
insertion_date       0
dtype: int64
                         title     price manufacturer   category fuel_type  \
0                      Audi S3  128000.0         Audi   Citadine    Petrol   
1  Mahindra Scorpio Pick-up DC   32000.0     Mahindra    Pick up    Diesel   
2                   Fiat Doblo   56000.0         Fiat  Monospace    Petrol   
3                  Ford Fusion   45000.0         Ford    Berline    Petrol   
4                MG GS AWD 2.0   61000.0           MG    SUV/4x4    Petrol   

   fiscal_horsepower transmission   mileage     year insertion_date  
0               19.0    Automatic  120000.0   6.2014     26.02.2024  
1                7.0       Manual   85000.0  11.2020     26.02.2024  
2                6.0       Manual   32000.0   7.2021     26.02.2024  
3       

Next we will convert some columns into their right data types

In [47]:
data["price"] = data["price"].astype(float)
data["insertion_date"] = pd.to_datetime(data["insertion_date"], format="%d.%m.%Y")

In [48]:
print(data.head())

                         title     price manufacturer   category fuel_type  \
0                      Audi S3  128000.0         Audi   Citadine    Petrol   
1  Mahindra Scorpio Pick-up DC   32000.0     Mahindra    Pick up    Diesel   
2                   Fiat Doblo   56000.0         Fiat  Monospace    Petrol   
3                  Ford Fusion   45000.0         Ford    Berline    Petrol   
4                MG GS AWD 2.0   61000.0           MG    SUV/4x4    Petrol   

   fiscal_horsepower transmission   mileage     year insertion_date  
0               19.0    Automatic  120000.0   6.2014     2024-02-26  
1                7.0       Manual   85000.0  11.2020     2024-02-26  
2                6.0       Manual   32000.0   7.2021     2024-02-26  
3                9.0       Manual  200000.0  11.2015     2024-02-26  
4               13.0    Automatic  100000.0   6.2017     2024-02-26  


Handle the "year" column to only keep the year because we're not interested in the month part

In [58]:
data["year"] = data["year"].astype(str)
data["year"] = data["year"].str.split(".").str[1].astype(int)

In [59]:
print(data.head())

                         title     price manufacturer   category fuel_type  \
0                      Audi S3  128000.0         Audi   Citadine    Petrol   
1  Mahindra Scorpio Pick-up DC   32000.0     Mahindra    Pick up    Diesel   
2                   Fiat Doblo   56000.0         Fiat  Monospace    Petrol   
3                  Ford Fusion   45000.0         Ford    Berline    Petrol   
4                MG GS AWD 2.0   61000.0           MG    SUV/4x4    Petrol   

   fiscal_horsepower transmission   mileage  year insertion_date  
0               19.0    Automatic  120000.0  2014     2024-02-26  
1                7.0       Manual   85000.0   202     2024-02-26  
2                6.0       Manual   32000.0  2021     2024-02-26  
3                9.0       Manual  200000.0  2015     2024-02-26  
4               13.0    Automatic  100000.0  2017     2024-02-26  


In [ ]:
filtered_data = data[data["year"].astype(str) == '202']

count = filtered_data.shape[0]

print(f"Number of rows with 'Year' exactly equal to '202': {count}")

Number of rows with 'Year' exactly equal to '202': 172


We saw that after formatting the year as we want, some values didn't fit exactly into the format we wanted as they lack 0 at the end, we find that '2020' values bacame '202', that's what we will fix in the next section

In [62]:
data["year"] = data["year"].astype(str).replace('202', '2020')
print(data.head())

                         title     price manufacturer   category fuel_type  \
0                      Audi S3  128000.0         Audi   Citadine    Petrol   
1  Mahindra Scorpio Pick-up DC   32000.0     Mahindra    Pick up    Diesel   
2                   Fiat Doblo   56000.0         Fiat  Monospace    Petrol   
3                  Ford Fusion   45000.0         Ford    Berline    Petrol   
4                MG GS AWD 2.0   61000.0           MG    SUV/4x4    Petrol   

   fiscal_horsepower transmission   mileage  year insertion_date  
0               19.0    Automatic  120000.0  2014     2024-02-26  
1                7.0       Manual   85000.0  2020     2024-02-26  
2                6.0       Manual   32000.0  2021     2024-02-26  
3                9.0       Manual  200000.0  2015     2024-02-26  
4               13.0    Automatic  100000.0  2017     2024-02-26  


In this next part we will focus on the category column, to make sure that our categorical columns are ready before moving on to feature engineering part

In [63]:
print(data["category"].unique())

['Citadine' 'Pick up' 'Monospace' 'Berline' 'SUV/4x4' 'Compacte'
 'Utilitaire' 'Coupé' 'Autres' 'Cabriolet']


In [64]:
data["category"] = data["category"].replace({
    'Citadine': 'City',
    'Pick up': 'Pickup',
    'Monospace': 'Minivan',
    'Berline': 'Sedan',
    'SUV/4x4': 'SUV',
    'Compacte': 'Compact',
    'Utilitaire': 'Utility',
    'Coupé': 'Coupe',
    'Autres': 'Other',
    'Cabriolet': 'Convertible'
})

In [65]:
print(data.head())

                         title     price manufacturer category fuel_type  \
0                      Audi S3  128000.0         Audi     City    Petrol   
1  Mahindra Scorpio Pick-up DC   32000.0     Mahindra   Pickup    Diesel   
2                   Fiat Doblo   56000.0         Fiat  Minivan    Petrol   
3                  Ford Fusion   45000.0         Ford    Sedan    Petrol   
4                MG GS AWD 2.0   61000.0           MG      SUV    Petrol   

   fiscal_horsepower transmission   mileage  year insertion_date  
0               19.0    Automatic  120000.0  2014     2024-02-26  
1                7.0       Manual   85000.0  2020     2024-02-26  
2                6.0       Manual   32000.0  2021     2024-02-26  
3                9.0       Manual  200000.0  2015     2024-02-26  
4               13.0    Automatic  100000.0  2017     2024-02-26  


Next, we move to feature engineering

First, we want to extract the age of each listed car

In [70]:
current_year = pd.to_datetime("today").year
data["age"] = current_year - data["year"].astype(int)

In [71]:
print(data.head())

                         title     price manufacturer category fuel_type  \
0                      Audi S3  128000.0         Audi     City    Petrol   
1  Mahindra Scorpio Pick-up DC   32000.0     Mahindra   Pickup    Diesel   
2                   Fiat Doblo   56000.0         Fiat  Minivan    Petrol   
3                  Ford Fusion   45000.0         Ford    Sedan    Petrol   
4                MG GS AWD 2.0   61000.0           MG      SUV    Petrol   

   fiscal_horsepower transmission   mileage  year insertion_date  age  
0               19.0    Automatic  120000.0  2014     2024-02-26   10  
1                7.0       Manual   85000.0  2020     2024-02-26    4  
2                6.0       Manual   32000.0  2021     2024-02-26    3  
3                9.0       Manual  200000.0  2015     2024-02-26    9  
4               13.0    Automatic  100000.0  2017     2024-02-26    7  


Next, we want to extract the price per km and per hosepower for each listed car

In [72]:
data["price_per_km"] = data["price"] / data["mileage"]
data["price_per_horsepower"] = data["price"] / data["fiscal_horsepower"]

In [73]:
print(data.head())

                         title     price manufacturer category fuel_type  \
0                      Audi S3  128000.0         Audi     City    Petrol   
1  Mahindra Scorpio Pick-up DC   32000.0     Mahindra   Pickup    Diesel   
2                   Fiat Doblo   56000.0         Fiat  Minivan    Petrol   
3                  Ford Fusion   45000.0         Ford    Sedan    Petrol   
4                MG GS AWD 2.0   61000.0           MG      SUV    Petrol   

   fiscal_horsepower transmission   mileage  year insertion_date  age  \
0               19.0    Automatic  120000.0  2014     2024-02-26   10   
1                7.0       Manual   85000.0  2020     2024-02-26    4   
2                6.0       Manual   32000.0  2021     2024-02-26    3   
3                9.0       Manual  200000.0  2015     2024-02-26    9   
4               13.0    Automatic  100000.0  2017     2024-02-26    7   

   price_per_km  price_per_horsepower  
0      1.066667           6736.842105  
1      0.376471         

We will remove the Insertion Date column from our dataset, because it won't be useful for our study

In [75]:
data.drop(['insertion_date'], axis=1)

,title,price,manufacturer,category,fuel_type,fiscal_horsepower,transmission,mileage,year,age,price_per_km,price_per_horsepower
0,Audi S3,128000.0,Audi,City,Petrol,19.0,Automatic,120000.0,2014,10,1.066667,6736.842105
1,Mahindra Scorpio Pick-up DC,32000.0,Mahindra,Pickup,Diesel,7.0,Manual,85000.0,2020,4,0.376471,4571.428571
2,Fiat Doblo,56000.0,Fiat,Minivan,Petrol,6.0,Manual,32000.0,2021,3,1.750000,9333.333333
3,Ford Fusion,45000.0,Ford,Sedan,Petrol,9.0,Manual,200000.0,2015,9,0.225000,5000.000000
4,MG GS AWD 2.0,61000.0,MG,SUV,Petrol,13.0,Automatic,100000.0,2017,7,0.610000,4692.307692
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,Renault Clio,28900.0,Renault,City,Petrol,5.0,Manual,233000.0,201,1823,0.124034,5780.000000
1595,KIA Sportage,100000.0,KIA,SUV,Diesel,10.0,Automatic,130000.0,2016,8,0.769231,10000.000000
1596,Volkswagen Golf 7 IQ drive,79500.0,Volkswagen,Compact,Petrol,6.0,Manual,100000.0,2019,5,0.795000,13250.000000
1597,Toyota Aygo,33000.0,Toyota,Compact,Petrol,4.0,Manual,69000.0,2020,4,0.478261,8250.000000


Wa want to get a general overview on the data we have, so we will get the statistical details of the whole dataset and for our relevant categorical attributes

In [76]:
numerical_stats = data.describe()
print(numerical_stats)

               price  fiscal_horsepower        mileage  \
count    1578.000000        1578.000000    1578.000000   
mean    89768.449937           8.242079  116409.249683   
min     10000.000000           4.000000      20.000000   
25%     44125.000000           6.000000   60000.000000   
50%     68000.000000           7.000000  101000.000000   
75%    110000.000000           9.000000  160000.000000   
max    748000.000000          43.000000  730000.000000   
std     72558.423950           4.542306   77428.368823   

                      insertion_date          age  price_per_km  \
count                           1578  1578.000000   1578.000000   
mean   2024-02-18 22:00:27.376425984    69.569075      8.073312   
min              2024-01-19 00:00:00     0.000000      0.033333   
25%              2024-02-16 00:00:00     4.000000      0.327801   
50%              2024-02-24 00:00:00     6.000000      0.694121   
75%              2024-02-26 00:00:00    10.000000      1.723887   
max     

In [79]:
categorical_columns = ["category", "manufacturer", "fuel_type"]

for col in categorical_columns:
    print(f"Value counts for {col} column:")
    print(data[col].value_counts())
    print("\n")

Value counts for category column:
category
SUV            463
Sedan          419
City           330
Compact        111
Coupe           96
Other           72
Utility         39
Pickup          28
Minivan         13
Convertible      7
Name: count, dtype: int64


Value counts for manufacturer column:
manufacturer
Mercedes      204
Volkswagen    161
BMW           154
KIA           114
Peugeot        94
Audi           71
Ford           58
Land Rover     58
Hyundai        52
Seat           51
Renault        49
Citroën        43
Porsche        40
Fiat           39
Nissan         37
Toyota         32
Haval          29
Mazda          29
MG             24
Chery          21
Ssangyong      18
Mahindra       18
Mini           17
Dacia          16
Chevrolet      16
Suzuki         14
Jeep           12
Jaguar         12
Skoda          11
Dongfeng        8
Smart           8
Opel            8
Alfa Romeo      8
Honda           7
Mitsubishi      6
Great Wall      5
Isuzu           4
Cupra           4
Othe

Correlation Matrix for Numerical Features to understand how the numerical features correlate with each other

In [ ]:
correlation_matrix = data.corr(numeric_only=True)

print(correlation_matrix)

                         price  fiscal_horsepower   mileage       age  \
price                 1.000000           0.521966 -0.351628 -0.097558   
fiscal_horsepower     0.521966           1.000000  0.073579  0.117493   
mileage              -0.351628           0.073579  1.000000  0.215787   
age                  -0.097558           0.117493  0.215787  1.000000   
price_per_km          0.030682          -0.001917 -0.075759 -0.010980   
price_per_horsepower  0.763935          -0.014557 -0.519041 -0.183325   

                      price_per_km  price_per_horsepower  
price                     0.030682              0.763935  
fiscal_horsepower        -0.001917             -0.014557  
mileage                  -0.075759             -0.519041  
age                      -0.010980             -0.183325  
price_per_km              1.000000              0.041524  
price_per_horsepower      0.041524              1.000000  


Save the cleaned dataset

In [82]:
data.to_csv("../data/cleaned_dataset.csv", index=False)

We will use the Min-Max Normalization four our numerical features

In [83]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[["price", "mileage", "fiscal_horsepower"]] = scaler.fit_transform(data[["price", "mileage", "fiscal_horsepower"]])


In [84]:
print(data.head())

                         title     price manufacturer category fuel_type  \
0                      Audi S3  0.159892         Audi     City    Petrol   
1  Mahindra Scorpio Pick-up DC  0.029810     Mahindra   Pickup    Diesel   
2                   Fiat Doblo  0.062331         Fiat  Minivan    Petrol   
3                  Ford Fusion  0.047425         Ford    Sedan    Petrol   
4                MG GS AWD 2.0  0.069106           MG      SUV    Petrol   

   fiscal_horsepower transmission   mileage  year insertion_date  age  \
0           0.384615    Automatic  0.164361  2014     2024-02-26   10   
1           0.076923       Manual  0.116414  2020     2024-02-26    4   
2           0.051282       Manual  0.043809  2021     2024-02-26    3   
3           0.128205       Manual  0.273953  2015     2024-02-26    9   
4           0.230769    Automatic  0.136963  2017     2024-02-26    7   

   price_per_km  price_per_horsepower  
0      1.066667           6736.842105  
1      0.376471         

Save Normalized dataset

In [85]:
data.to_csv("../data/normalized_dataset.csv", index=False)